In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import pandas as pd
from tensorflow.keras.callbacks import Callback

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

import tensorflow.keras.applications

from pathlib import Path

#from tf_explain.core.grad_cam import GradCAM

import cv2

import math
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import DenseNet121,DenseNet169,  \
                                          DenseNet201,InceptionResNetV2,  \
                                          InceptionV3,MobileNet,MobileNetV2,  \
                                          NASNetLarge,NASNetMobile,ResNet101,  \
                                          ResNet101V2,ResNet152,ResNet152V2,  \
                                          ResNet50,ResNet50V2,VGG16,VGG19,Xception 

from tensorflow.keras.metrics import Accuracy, Precision, Recall, \
                                     FalsePositives, FalsePositives, \
                                     TruePositives, TrueNegatives

from datetime import datetime


IMG_HEIGHT = 512
IMG_WIDTH = 512
EPOCHS = 6

CHANNELS =3
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)

FILTER_SIZE = 3

lrs = [('6e-4',6e-4), ('1e-3',1e-3), ('6e-3',6e-3)]
batch_sizes = [('8',8),('16',16)]
BATCH_SIZE = batch_sizes[0][1]
optimizers = ['Adam', 'Nadam', 'SGD']
opt = 'SGD'

l = lrs[1]
lr = l[1]

mode = 'mixed'

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_visible_devices(gpus[1], 'GPU')

In [3]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [4]:
"""
Module for Image Save
"""


def save_grayscale(image, output_dir, output_name):
    """
    Save a 3D Numpy array (H, W, 1) as an image.
    Args:
        image (numpy.ndarray): Image to save
        output_dir (str): Output directory
        output_name (str): Output name
    """
    Path.mkdir(Path(output_dir), parents=True, exist_ok=True)

    cv2.imwrite(str(Path(output_dir) / output_name), image)


def save_rgb(image, output_dir, output_name):
    """
    Save a 3D Numpy array (H, W, 3) as an image.
    Args:
        image (numpy.ndarray): Image to save
        output_dir (str): Output directory
        output_name (str): Output name
    """
    Path.mkdir(Path(output_dir), parents=True, exist_ok=True)

    cv2.imwrite(
        str(Path(output_dir) / output_name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    )

In [5]:
def grid_display(array, num_rows=None, num_columns=None):
    """
    Display a list of images as a grid.
    Args:
        array (numpy.ndarray): 4D Tensor (batch_size, height, width, channels)
    Returns:
        numpy.ndarray: 3D Tensor as concatenation of input images on a grid
    """
    if num_rows is not None and num_columns is not None:
        total_grid_size = num_rows * num_columns
        if total_grid_size < len(array):
            warnings.warn(
                Warning(
                    "Given values for num_rows and num_columns doesn't allow to display "
                    "all images. Values have been overrided to respect at least num_columns"
                )
            )
            num_rows = math.ceil(len(array) / num_columns)
    elif num_rows is not None:
        num_columns = math.ceil(len(array) / num_rows)
    elif num_columns is not None:
        num_rows = math.ceil(len(array) / num_columns)
    else:
        num_rows = math.ceil(math.sqrt(len(array)))
        num_columns = math.ceil(math.sqrt(len(array)))

    number_of_missing_elements = num_columns * num_rows - len(array)
    # We fill the array with np.zeros elements to obtain a perfect square
    array = np.append(
        array,
        np.zeros((number_of_missing_elements, *array[0].shape)).astype(array.dtype),
        axis=0,
    )

    grid = np.concatenate(
        [
            np.concatenate(
                array[index * num_columns : (index + 1) * num_columns], axis=1
            )
            for index in range(num_rows)
        ],
        axis=0,
    )

    return grid


def filter_display(array, num_rows=None, num_columns=None):
    """
    Display a list of filter outputs as a greyscale images grid.
    Args:
        array (numpy.ndarray): 4D Tensor (batch_size, height, width, channels)
    Returns:
        numpy.ndarray: 3D Tensor as concatenation of input images on a grid
    """
    return grid_display(
        np.concatenate(np.rollaxis(array, 3, 1), axis=0), num_rows, num_columns
    )


def image_to_uint_255(image):
    """
    Convert float images to int 0-255 images.
    Args:
        image (numpy.ndarray): Input image. Can be either [0, 255], [0, 1], [-1, 1]
    Returns:
        numpy.ndarray:
    """
    if image.dtype == np.uint8:
        return image

    if image.min() < 0:
        image = (image + 1.0) / 2.0

    return (image * 255).astype("uint8")


def heatmap_display(
    heatmap, original_image, colormap=cv2.COLORMAP_VIRIDIS, image_weight=0.7
):
    """
    Apply a heatmap (as an np.ndarray) on top of an original image.
    Args:
        heatmap (numpy.ndarray): Array corresponding to the heatmap
        original_image (numpy.ndarray): Image on which we apply the heatmap
        colormap (int): OpenCV Colormap to use for heatmap visualization
        image_weight (float): An optional `float` value in range [0,1] indicating the weight of
            the input image to be overlaying the calculated attribution maps. Defaults to `0.7`
    Returns:
        np.ndarray: Original image with heatmap applied
    """
    heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))

    image = image_to_uint_255(original_image)

    heatmap = (heatmap - np.min(heatmap)) / (heatmap.max() - heatmap.min())

    heatmap = cv2.applyColorMap(
        cv2.cvtColor((heatmap * 255).astype("uint8"), cv2.COLOR_GRAY2BGR), colormap
    )

    output = cv2.addWeighted(
        cv2.cvtColor(image, cv2.COLOR_RGB2BGR), image_weight, heatmap, 1, 0
    )

    return cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

In [6]:
"""
Core Module for Grad CAM Algorithm
"""

class GradCAM:

    """
    Perform Grad CAM algorithm for a given input
    Paper: [Grad-CAM: Visual Explanations from Deep Networks
            via Gradient-based Localization](https://arxiv.org/abs/1610.02391)
    """

    def explain(
        self,
        validation_data,
        model,
        class_index,
        layer_name=None,
        colormap=cv2.COLORMAP_VIRIDIS,
        image_weight=0.7,
    ):
        """
        Compute GradCAM for a specific class index.
        Args:
            validation_data (Tuple[np.ndarray, Optional[np.ndarray]]): Validation data
                to perform the method on. Tuple containing (x, y).
            model (tf.keras.Model): tf.keras model to inspect
            class_index (int): Index of targeted class
            layer_name (str): Targeted layer for GradCAM. If no layer is provided, it is
                automatically infered from the model architecture.
            colormap (int): OpenCV Colormap to use for heatmap visualization
            image_weight (float): An optional `float` value in range [0,1] indicating the weight of
                the input image to be overlaying the calculated attribution maps. Defaults to `0.7`.
        Returns:
            numpy.ndarray: Grid of all the GradCAM
        """
        images, _ = validation_data

        if layer_name is None:
            layer_name = self.infer_grad_cam_target_layer(model)

        outputs, guided_grads = GradCAM.get_gradients_and_filters(
            model, images, layer_name, class_index
        )

        cams = GradCAM.generate_ponderated_output(outputs, guided_grads)

        heatmaps = np.array(
            [
                # not showing the actual image if image_weight=0
                heatmap_display(cam.numpy(), image, colormap, image_weight)
                for cam, image in zip(cams, images)
            ]
        )

        grid = grid_display(heatmaps)

        return grid

    @staticmethod
    def infer_grad_cam_target_layer(model):
        """
        Search for the last convolutional layer to perform Grad CAM, as stated
        in the original paper.
        Args:
            model (tf.keras.Model): tf.keras model to inspect
        Returns:
            str: Name of the target layer
        """
        for layer in reversed(model.layers):
            # Select closest 4D layer to the end of the network.
            if len(layer.output_shape) == 4:
                print(layer.name)
                return layer.name

        raise ValueError(
            "Model does not seem to contain 4D layer. Grad CAM cannot be applied."
        )

    @staticmethod
    @tf.function
    def get_gradients_and_filters(model, images, layer_name, class_index):
        """
        Generate guided gradients and convolutional outputs with an inference.
        Args:
            model (tf.keras.Model): tf.keras model to inspect
            images (numpy.ndarray): 4D-Tensor with shape (batch_size, H, W, 3)
            layer_name (str): Targeted layer for GradCAM
            class_index (int): Index of targeted class
        Returns:
            Tuple[tf.Tensor, tf.Tensor]: (Target layer outputs, Guided gradients)
        """
        grad_model = tf.keras.models.Model(
            [model.inputs], [model.get_layer(layer_name).output, model.output]
        )

        with tf.GradientTape() as tape:
            inputs = tf.cast(images, tf.float32)
            conv_outputs, predictions = grad_model(inputs)
            loss = predictions[:, class_index]

        grads = tape.gradient(loss, conv_outputs)
        

        guided_grads = (
            tf.cast(conv_outputs > 0, "float32") * tf.cast(grads > 0, "float32") * tf.cast(grads,"float32")
        )

        return conv_outputs, guided_grads

    @staticmethod
    def generate_ponderated_output(outputs, grads):
        """
        Apply Grad CAM algorithm scheme.
        Inputs are the convolutional outputs (shape WxHxN) and gradients (shape WxHxN).
        From there:
            - we compute the spatial average of the gradients
            - we build a ponderated sum of the convolutional outputs based on those averaged weights
        Args:
            output (tf.Tensor): Target layer outputs, with shape (batch_size, Hl, Wl, Nf),
                where Hl and Wl are the target layer output height and width, and Nf the
                number of filters.
            grads (tf.Tensor): Guided gradients with shape (batch_size, Hl, Wl, Nf)
        Returns:
            List[tf.Tensor]: List of ponderated output of shape (batch_size, Hl, Wl, 1)
        """

        maps = [
            GradCAM.ponderate_output(output, grad)
            for output, grad in zip(outputs, grads)
        ]

        return maps

    @staticmethod
    def ponderate_output(output, grad):
        """
        Perform the ponderation of filters output with respect to average of gradients values.
        Args:
            output (tf.Tensor): Target layer outputs, with shape (Hl, Wl, Nf),
                where Hl and Wl are the target layer output height and width, and Nf the
                number of filters.
            grads (tf.Tensor): Guided gradients with shape (Hl, Wl, Nf)
        Returns:
            tf.Tensor: Ponderated output of shape (Hl, Wl, 1)
        """
        weights = tf.reduce_mean(grad, axis=(0, 1))

        # Perform ponderated sum : w_i * output[:, :, i]
        cam = tf.reduce_sum(tf.multiply(tf.cast(weights,"float32"), tf.cast(output,'float32') ), axis=-1)

        return cam

    def save(self, grid, output_dir, output_name):
        """
        Save the output to a specific dir.
        Args:
            grid (numpy.ndarray): Grid of all the heatmaps
            output_dir (str): Output directory path
            output_name (str): Output name
        """
        save_rgb(grid, output_dir, output_name)


In [7]:
train_df = pd.read_csv("./train-one-hot.csv")
train_df.head()

,image,normal,fracture
0,2.25.32275753493272066455728468222013315182.png,0,1
1,2.25.276430725003951681573364027711855344001.png,0,1
2,2.25.253422208237850690154351736536198463586.png,0,1
3,2.25.235981286495328419407334450332699270503.png,1,0
4,2.25.209179633594774313844201191440779232058.png,0,1


In [8]:
#reduce training set for quick iteration debugging
#train_df = train_df.head(10)

In [9]:
valid_df = pd.read_csv("./val-one-hot.csv")
valid_df.head(25)

,image,normal,fracture
0,2.25.266368834198878584063538399027795661402.png,0,1
1,2.25.198053970490389086906016595802053064438.png,0,1
2,2.25.132649454315604163163469913960889989839.png,1,0
3,2.25.141160371661672481971139116295496247657.png,0,1
4,2.25.190934592832011590665778618155747001337.png,0,1
5,2.25.85588580190228693739225557679679405803.png,0,1
6,2.25.304602438475911337153274891496014274303.png,1,0
7,2.25.114370779292559265124294169713118718955.png,1,0
8,2.25.166101061863990858418630862457202935915.png,1,0
9,2.25.257743718283809201969571483810430772338.png,0,1


In [10]:
valid_df.tail(100)

,image,normal,fracture
782,2.25.310656859401367861691413406710441338016.png,1,0
783,2.25.159014496226658261501698112987163403282.png,0,1
784,2.25.300379945108852291003294002596377791253.png,1,0
785,2.25.331675432825168281256094116304610068304.png,1,0
786,2.25.238514064404406518979636181721800247856.png,1,0
...,...,...,...
877,2.25.306015666503773354154907958287120514904.png,0,1
878,2.25.99487455393505948788675807627202878608.png,0,1
879,2.25.145872723669425894281076127033757907791.png,0,1
880,2.25.217558881674041455047024948366123645839.png,0,1


In [11]:
test_df = pd.read_csv("./test-one-hot.csv")
test_df.head()

,image,normal,fracture
0,2.25.173789258248807928758296515316743888964.png,0,1
1,2.25.307941367272363205123396420445524060815.png,0,1
2,2.25.21224089373954987773412215178159360714.png,1,0
3,2.25.227746943974749430406884127570205991943.png,0,1
4,2.25.225671999229605271677288817920080550497.png,1,0


In [12]:
y_test = test_df['fracture'].values

In [13]:
y_test

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,

In [14]:
#architecture = ('DenseNet201', 
                #DenseNet201(input_shape=IMG_SHAPE,include_top=False,weights='imagenet'),
                #'conv2_block1_1_conv',
                #'conv5_block32_2_conv')

architecture = ('ResNet152V2', 
                ResNet152V2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet'),
               'conv2_block1_1_conv',
               'conv5_block3_3_conv')

In [15]:
labels = ['normal','fracture']

In [16]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=8, seed=1, target_w = 512, target_h = 512):
       
    print("getting train generator...") 
    # normalize images
    image_generator = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=180,
        height_shift_range=[-10,10],
        width_shift_range=[-10,10],
        rescale=1./255)
    
    # flow from directory with specified batch size
    # and target image size
    generator = image_generator.flow_from_dataframe(
            dataframe=df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            target_size=(target_w,target_h))
    
    return generator

In [17]:
def get_test_and_valid_generator(valid_df, test_df, train_df, image_dir, x_col, y_cols, sample_size=2000, batch_size=8, seed=1, target_w = 512, target_h = 512):
    
    print("getting test and valid generators...")
    

    # use sample to fit mean and std for test set generator
    image_generator = ImageDataGenerator(
       rescale=1./255)
    
    

    # get test generator
    valid_generator = image_generator.flow_from_dataframe(
            dataframe=valid_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))

    test_generator = image_generator.flow_from_dataframe(
            dataframe=test_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))
    return valid_generator, test_generator

In [18]:
IMAGE_DIR = "/opt/AIStorage/PLAYGROUND/images/512/filtered/all_images"
train_generator = get_train_generator(train_df, IMAGE_DIR, "image", labels, batch_size=BATCH_SIZE)
valid_generator, test_generator= get_test_and_valid_generator(valid_df, test_df, train_df, IMAGE_DIR, "image", labels, batch_size=BATCH_SIZE)

getting train generator...
Found 7063 validated image filenames.
getting test and valid generators...
Found 882 validated image filenames.
Found 882 validated image filenames.


In [19]:
batch = valid_generator.next()

In [20]:
batch[1].shape

(8, 2)

In [21]:
def create_model():
    base_model_x = None
    model_x=None
    
    base_model_x = architecture[1]
    
    #base_model.trainable = False
        
    for layer in base_model_x.layers:
        if layer.name.endswith('bn'):
            layer.trainable = True
        else:
            layer.trainable = False
                
    x = base_model_x.output
        
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
        
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(512,activation='relu')(x)
        
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(2, activation='softmax', dtype=tf.float32)(x)
        
    model_x = Model(inputs=base_model_x.input,outputs=predictions)
    
    return model_x
    

In [22]:
valid_examples = [ ('2.25.312225116666381863237801067474791146955.png',0),
                   ('2.25.331215779585395792556605116028426521840.png',0),
                   ('2.25.294885570448455301317418340729685335337.png',1),
                   ('2.25.29996689003180237805319604864278729987.png',1),
                   ('2.25.77133591367680957046388157253278169022.png',1)]

In [23]:
layer_names = [architecture[2], architecture[3]]
indexes = [0,1]

In [24]:
class SaveGradCAMCallback(Callback):
    
    def __init__(
        self,
        output_dir=None,
        layer_names=None,
        examples=None,
        indexes=None
    ):
        super(SaveGradCAMCallback,self).__init__()
        self.output_dir = output_dir
        self.layer_names = layer_names
        self.examples = examples
        self.indexes = indexes
        
    def on_epoch_end(self, epoch, logs=None):
       
        
        #self.model.save_weights(str(self.output_dir) + '/saved_models')
        
        #policy = mixed_precision.Policy('float32')
        
        #mixed_precision.set_policy(policy)
        
        
        #saved_model = create_model()
        
        saved_model = self.model
        
        #saved_model.load_weights(str(self.output_dir) + '/saved_models').expect_partial()
        
        
        
        for example in self.examples:
            for layer_name in self.layer_names:
                for index in self.indexes:
                    IMAGE_PATH = IMAGE_DIR + '/' + example[0]
        
                    img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(512, 512))
                    img = tf.keras.preprocessing.image.img_to_array(img)

                    img = img/255.0

                    data = ([img], None)

                    explainer = GradCAM()
                    
                    grid = explainer.explain(data, saved_model, class_index=index, image_weight=0.75, layer_name=layer_name)
    
                    file_id = example[0][:-4]
                    save_name = file_id + '--' + str(epoch) + '--' + layer_name + '-- ' + str(index) + '.png'
                    
                    save_location = self.output_dir / file_id / layer_name / str(index)
                    #explainer.save(grid, self.output_dir, save_name)
                    explainer.save(grid, save_location, save_name)
                    
        
        #policy = mixed_precision.Policy('mixed_float16')
        #mixed_precision.set_policy(policy)
        

In [25]:
        
'''        IMAGE_PATH = '/opt/AIStorage/PLAYGROUND/images/512/positive/2.25.181219916381488720424316063428743387152.png'
        img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(512, 512))
        img = tf.keras.preprocessing.image.img_to_array(img)

        img = img/255.0


        data = ([img], None)

        index = 0
        explainer = GradCAM()
        
        saved_model.trainable = True
        
        #layer ='conv2_block1_1_conv'
        layer ='conv5_block32_2_conv'
    
        grid = explainer.explain(data, saved_model, class_index=index, image_weight=0.5, layer_name=layer)
    
        explainer.save(grid, ".", "grad_cam10_16__2x0_5_adam.png")
        
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_policy(policy)
'''

'        IMAGE_PATH = \'/opt/AIStorage/PLAYGROUND/images/512/positive/2.25.181219916381488720424316063428743387152.png\'\n        img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(512, 512))\n        img = tf.keras.preprocessing.image.img_to_array(img)\n\n        img = img/255.0\n\n\n        data = ([img], None)\n\n        index = 0\n        explainer = GradCAM()\n        \n        saved_model.trainable = True\n        \n        #layer =\'conv2_block1_1_conv\'\n        layer =\'conv5_block32_2_conv\'\n    \n        grid = explainer.explain(data, saved_model, class_index=index, image_weight=0.5, layer_name=layer)\n    \n        explainer.save(grid, ".", "grad_cam10_16__2x0_5_adam.png")\n        \n        policy = mixed_precision.Policy(\'mixed_float16\')\n        mixed_precision.set_policy(policy)\n'

In [ ]:


STEPS_PER_EPOCH = len(train_generator) 
VAL_STEPS_PER_EPOCH = len(valid_generator) 

print("")
print("Arch: ", architecture[0])

optimizer = None
        
if opt == 'Adam':
    optimizer = tf.keras.optimizers.Adam(lr=lr)
elif opt == 'Nadam':
    optimizer = tf.keras.optimizers.Nadam(lr=lr)
else:
    optimizer = tf.keras.optimizers.SGD(lr=lr, momentum=0.9, decay=1e-6, nesterov=True)    
    
model = create_model()

#model.summary()
        
        
model.compile(optimizer=optimizer,loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),metrics=['acc'])

run_name = architecture[0] + '_' + l[0] + '_' + opt + '_' + str(BATCH_SIZE) + '_' + mode
run_dir = datetime.now().strftime("%Y%m%d-%H%M%S.%f") + '_' + run_name
output_dir = Path("./logs/grad_cam") / run_dir

Path.mkdir(Path(output_dir), parents=True, exist_ok=True)

history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=valid_generator, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_steps=VAL_STEPS_PER_EPOCH,
                    callbacks=[SaveGradCAMCallback(output_dir=output_dir,
                                                   layer_names=layer_names,
                                                   examples=valid_examples,
                                                   indexes=indexes)])
        
        
        
print("")
print("")



Arch:  DenseNet201
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 883 steps, validate for 111 steps
Epoch 1/6
883/883 [==============================] - 422s 478ms/step - loss: 0.8107 - acc: 0.6401 - val_loss: 1.1084 - val_acc: 0.6440
Epoch 2/6
883/883 [==============================] - 366s 415ms/step - loss: 0.6115 - acc: 0.7214 - val_loss: 0.4771 - val_acc: 0.8016
Epoch 3/6
520/883 [================>.............] - ETA: 2:26 - loss: 0.5278 - acc: 0.7730

In [ ]:
np.save(output_dir / 'history.npy', history.history)

In [ ]:
#loading the history object as a map
#history=np.load('history.npy',allow_pickle='TRUE').item()

In [ ]:
history.history

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the model: ", len(model.layers))

# Fine-tune from about the middle layer onwards
fine_tune_at = int(.5 * len(model.layers))

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
    if layer.name.endswith('bn'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
if opt == 'Adam':
    optimizer = tf.keras.optimizers.Adam(lr=lr/6)
elif opt == 'Nadam':
    optimizer = tf.keras.optimizers.Nadam(lr=lr/6)
else:
    optimizer = tf.keras.optimizers.SGD(lr=lr/6, momentum=0.9, decay=1e-6, nesterov=True)   
    
model.compile(optimizer=optimizer,loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['acc'])

In [ ]:
fine_tune_epochs = 8
total_epochs =  EPOCHS + fine_tune_epochs

In [ ]:
history_fine = model.fit(train_generator,
                    epochs=total_epochs,
                    initial_epoch =  history.epoch[-1],
                    validation_data=valid_generator, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_steps=VAL_STEPS_PER_EPOCH,
                    callbacks=[SaveGradCAMCallback(output_dir=output_dir,
                                                   layer_names=layer_names,
                                                   examples=valid_examples,
                                                   indexes=indexes)])

In [ ]:
np.save(output_dir / 'history_fine.npy', history_fine.history)

In [ ]:
acc += history_fine.history['acc']
val_acc += history_fine.history['val_acc']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.trainable = True

In [ ]:
if opt == 'Adam':
    optimizer = tf.keras.optimizers.Adam(lr=lr/10)
elif opt == 'Nadam':
    optimizer = tf.keras.optimizers.Nadam(lr=lr/10)
else:
    optimizer = tf.keras.optimizers.SGD(lr=lr/10, momentum=0.9, decay=1e-6, nesterov=True)
    
model.compile(optimizer=optimizer,loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['acc'])

In [ ]:
unfreeze_epochs = 8
total_epochs =  EPOCHS + fine_tune_epochs + unfreeze_epochs

In [ ]:
history_unfreeze = model.fit(train_generator,
                    epochs=total_epochs,
                    initial_epoch =  history_fine.epoch[-1],
                    validation_data=valid_generator, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    validation_steps=VAL_STEPS_PER_EPOCH,
                    callbacks=[SaveGradCAMCallback(output_dir=output_dir,
                                                   layer_names=layer_names,
                                                   examples=valid_examples,
                                                   indexes=indexes)])

In [ ]:
np.save(output_dir / 'history_unfreeze.npy', history_unfreeze.history)

In [ ]:
acc += history_unfreeze.history['acc']
val_acc += history_unfreeze.history['val_acc']

loss += history_unfreeze.history['loss']
val_loss += history_unfreeze.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.plot([(EPOCHS+fine_tune_epochs)-1,(EPOCHS+fine_tune_epochs)-1],
          plt.ylim(), label='Start Full Unfreeze')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.plot([(EPOCHS+fine_tune_epochs)-1,(EPOCHS+fine_tune_epochs)-1],
          plt.ylim(), label='Start Full Unfreeze')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
metrics = model.evaluate(test_generator)

In [ ]:
metrics

In [ ]:
np.save(output_dir / 'metrics.npy', metrics)

In [ ]:
preds_test = model.predict(test_generator)

In [ ]:
preds_test

In [ ]:
np.save(output_dir / 'preds_test.npy', preds_test)

In [ ]:
test_df = pd.read_csv("./test-one-hot.csv")
test_df.head()

In [ ]:
np.save(output_dir / 'y_test.npy', y_test)